In [1]:
from mass import MassClassifier
from scipy import sparse
import numpy as np
import pandas as pd

/home/home1/student/tianjian1/.conda/envs/ml/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
old=pd.read_csv("0.csv")

In [3]:
edges=[
        ("R1","feature15"),
        ("R1","feature13"),
        ("R2","feature13"),
        ("R2","feature19"),
        ("R2","feature61_0"),
        ("R2","feature61_1"),
        ("R2","feature61_2"), 
        ("R2","feature61_3"),
        ("R2","feature61_4"),
        ("R2","feature61_5"),
        ("R2","feature61_6"),
        ("R2","feature61_7"),
        ("R2","feature69_0"), 
        ("R2","feature69_1"),
        ("R2","feature69_2"),
        ("R2","feature69_3"),
        ("R2","feature69_4"), 
        ("R2","feature69_5"),
        ("R2","feature69_6"), 
        ("R2","feature28_0"), 
        ("R2","feature28_1"),
        ("R2","feature28_2"), 
        ("R2","feature28_3"),
        ("R2","feature28_4"),
        ("R2","feature28_5"),
        ("R2","feature28_6"),
        ("R2","feature28_7"),
        ("R2","feature36_0"),
        ("R2","feature36_1"),
        ("R2","feature36_2"), 
        ("R2","feature36_3"),
        ("R2","feature36_4"), 
        ("R2","feature36_5"),
        ("R2","feature36_6"),
        ("R2","feature36_7"),
        ("R3","feature60"),
        ("R3","feature61_0"),
        ("R3","feature61_1"),
        ("R3","feature61_2"),
        ("R3","feature61_3"),
        ("R3","feature61_4"),
        ("R3","feature61_5"),
        ("R3","feature61_6"),
        ("R3","feature61_7"),
        ("R3","feature69_0"),
        ("R3","feature69_1"),
        ("R3","feature69_2"),
        ("R3","feature69_3"),
        ("R3","feature69_4"),
        ("R3","feature69_5"),
        ("R3","feature69_6"),
        ("R3","feature20_0"),
        ("R3","feature20_1"),
        ("R3","feature20_2"),
        ("R3","feature20_3"),
        ("R3","feature20_4"),
        ("R3","feature20_5"),
        ("R3","feature20_6"),
        ("R3","feature20_7"),
        ("R3","feature28_0"),
        ("R3","feature28_1"),
        ("R3","feature28_2"),
        ("R3","feature28_3"),
        ("R3","feature28_4"),
        ("R3","feature28_5"),
        ("R3","feature28_6"),
        ("R3","feature28_7"),
        ("R3","feature36_0"),
        ("R3","feature36_1"),
        ("R3","feature36_2"),
        ("R3","feature36_3"),
        ("R3","feature36_4"),
        ("R3","feature36_5"),
        ("R3","feature36_6"),
        ("R3","feature36_7"),        
    ]

In [4]:
traindata=np.load("train_data.npy")
testdata=np.load("test_data.npy")
y=np.load("train_label.npy")
test_label=np.load("test_label.npy")
X1=traindata[(~(y==0).all(1))][:,2,:]
X2=traindata[((y==0).all(1))][:,2,:]
Y1=y[(~(y==0).all(1))]
mytrainfeature=pd.DataFrame(data= X1, columns=old.columns[1:])
mytrainlabel=pd.DataFrame(data= Y1, columns=["R1","R2","R3"])

In [5]:
featurelist=list(set([i[1] for i in edges]))

In [6]:
mytotaldata=np.concatenate([X1,X2,testdata[:,2,:]],axis=0)
mytotalfeature=pd.DataFrame(data= mytotaldata, columns=old.columns[1:])

In [7]:
for i in mytotalfeature.columns:
    if len(pd.unique(mytotalfeature[i]))>20:
        mytotalfeature[i]=pd.cut(mytotalfeature[i],bins=10)
        mytotalfeature[i]=mytotalfeature[i].cat.codes

In [8]:
labeleddata=pd.concat([mytotalfeature[featurelist][:1407],mytrainlabel],axis=1)

In [9]:
testdf=mytotalfeature[featurelist][-600:]
unlabeldf=mytotalfeature[featurelist][1407:-600]

In [10]:
unlabeldf=unlabeldf.reset_index(drop=True)

In [11]:
unlabeldf=unlabeldf.drop([838])

In [12]:
from pgmpy.models import BayesianNetwork

telecom_model = BayesianNetwork(
  edges
)


In [13]:
telecom_model.fit(labeleddata)
#telecom_model.get_cpds()

/home/home1/student/tianjian1/.conda/envs/ml/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [14]:
#telecom_model.get_cpds()

# EM algorithm

In [15]:
for EMiter in range(50):
    #E step
    MCy=[]
    N_MC=20
    for i in range(N_MC):
        MCy.append(telecom_model.predict(unlabeldf,stochastic=True))
    MCx=[]
    MClabeled=[]
    for i in range(N_MC):
        MCx.append(unlabeldf)
        MClabeled.append(labeleddata)

    MCX=pd.concat(MCx,axis=0)
    MCY=pd.concat(MCy,axis=0)
    if EMiter!=0:
        Ydiffnorm=np.linalg.norm(MCY.values-MCY_old.values)
        print("iter:",EMiter,"diff:",Ydiffnorm)
        if Ydiffnorm<5:
            break
    
    MCY_old=MCY.copy()
    MCLabeled=pd.concat(MClabeled,axis=0)
    MCUlabeled=pd.concat([MCX,MCY],axis=1)
    data=pd.concat([MCLabeled,MCUlabeled],axis=0)    

    #M step
    telecom_model.fit(data)   
        
     


iter: 1 diff: 41.255302689472536



iter: 2 diff: 19.157244060668017



iter: 3 diff: 143.2550173641398



iter: 4 diff: 16.3707055437449



iter: 5 diff: 16.06237840420901


...


iter: 44 diff: 16.217274740226856



iter: 45 diff: 15.84297951775486



iter: 46 diff: 16.792855623746664



iter: 47 diff: 16.1245154965971


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
y_pred2=telecom_model.predict(testdf,stochastic=False)

In [17]:
def score(pred,label):
    plus=np.sum(pred*label,axis=1)
    minus=np.sum(pred*(1-label),axis=1)
    return np.mean((plus-minus)/np.sum(label,axis=1))
score(y_pred2[["R1","R2","R3"]].values,test_label)

0.6411111111111112